In [2]:
import numpy as np

In [3]:
# Had a little help as always
# https://www.geeksforgeeks.org/dice-throw-problem/

# Here is the table of correct answers for 6 sided dice, to check accuracy
# https://wizardofodds.com/gambling/dice/2/
def find_ways(k,n,x):
    if x<1: return 0
    if n==1: return int(x<=k)#1 if true else 0
    num_ways = 0
    for i in range(1,k+1):
        num_ways += find_ways(k,n-1,x-i)
    return num_ways

In [4]:
find_ways(6,5,14)

540

In [5]:
'''
Dynamic Solution -- translated to numpy
'''
def findWays(k,n,x):
    # The extra row/col make indexing easier, but could be removed (it's not really too bad tho)
    table = np.zeros((n+1,x+1), dtype=np.uint64)
    
    if k < x:
        table[1][1:k+1] = 1
    else:
        table[1][1:x+1] = 1
    
    
    for i in range(2,n+1):
        for j in range(2,x+1):
            l = 1
            while l<j and l<=k:
                table[i][j] += table[i-1][j-l]
                l+=1

                
    print(table)
    return table[n][x]

    

In [6]:
# findWays(6,6,6)


In [7]:
def get_freq(k,n):
    x = n*k
    table = np.zeros((n+1,x+1), dtype = np.float64)
    
    if k < x:
        table[1][:k+1] = 1
    else:
        table[1][:x+1] = 1
    
    
    for i in range(2,n+1):
        for j in range(2,x+1):
            #table[i][j] = table[i][j-1] + table[i-1][j-1]
            l = 1
            while l<j and l<=k:
                table[i][j] += table[i-1][j-l]
                l+=1
#    print(table)
    
    return table[n]

In [8]:
get_freq(6,3)

array([ 0.,  0.,  0.,  1.,  3.,  6., 10., 15., 21., 25., 27., 27., 25.,
       21., 15., 10.,  6.,  3.,  1.])

In [9]:
def get_list_freq(dice):
    #setup
    #there are n dice, like last time
    n = dice.size
    # x needs to be the upper bound, which would be rolling k on all dice
    x = np.sum(dice)
    # init the table
    table = np.zeros((n+1,x+1), dtype=np.uint64)
    
    dice = sorted(dice)[::-1]
    
    if np.max(dice) < x:
        table[1][:np.max(dice)+1] = 1
    else:
        table[1][:x+1] = 1
    
    i = 1
    for k in dice:
        # we need to handle each dice face independently
        #might need temporary bound for 2nd loop
        for j in range(2,x+1):
            l = 1
            while l<j and l <=k:
                table[i][j] += table[i-1][j-l]
                l+=1   
        i += 1
#    print(table)
    return table[n]

In [10]:
print(get_list_freq(np.array([6,6,3])))
print(get_freq(6,3))

[ 0  0  0  1  3  6  9 12 15 16 15 12  9  6  3  1]
[ 0.  0.  0.  1.  3.  6. 10. 15. 21. 25. 27. 27. 25. 21. 15. 10.  6.  3.
  1.]


In [11]:
def get_dict_freq(some_dict):
    lis = []
    for key in some_dict:
        lis += [key]*some_dict[key]
    to_call = np.asarray(lis)
    # print(to_call)
    return get_list_freq(to_call)

In [12]:
test = {6:2, 4:1}
print(get_dict_freq(test))
print(get_list_freq(np.array([6,6,4])))

[ 0  0  0  1  3  6 10 14 18 20 20 18 14 10  6  3  1]
[ 0  0  0  1  3  6 10 14 18 20 20 18 14 10  6  3  1]


In [13]:
# np.random.choice(5000,200,replace=False)

In [14]:
'''
Attempting to use linear algebra to solve get_freq(6,3), then will build from there

size = nk-n+1
'''
size = 3*6-3+1


Y = np.ones((size,size))
Y = np.triu(Y)

Y = np.triu(np.flip(Y,1))

'''
I also need to initialize the starting array, a 1xSize arry with the initial frrequencies
'''

a = np.zeros(size)
a[0:6] = 1
#print(a)
print('The transformation matrix A:')
# print(Y)

for i in range(size-2*6):
    Y[i][(6+i):size-(6+i)] = 0
    

print(Y)

print(a)

print(np.linalg.matrix_power(Y.T,2).dot(a))


The transformation matrix A:
[[1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0.]
 [0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0.]
 [0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[ 1.  3.  6. 10. 15. 21. 25. 27. 27. 25. 21. 15. 10.  6.  3.  1.]


In [15]:
def freq(k,n):
    size = k*n-n+1
    # print(size)
    X = np.ones((size,size))
    X = np.triu(X)
    X = np.triu(np.flip(X,1))
    for i in range(size-2*k):
        X[i][k+i:size-(k+i)] = 0
    # print(X.T)
    a = np.zeros(size)
    a[0:k] = 1
    # print(a)
    return np.linalg.matrix_power(X.T,n-1).dot(a)
    #return X

In [16]:
result = get_freq(6, 300)
# %timeit get_freq(6, 300)
result[(300*5)//2+300]

3.7588197132593194e+231

In [17]:
res = freq(6, 3)
# %timeit freq(6, 300)
#res[(300*5)//2]
print(res)

[ 1.  3.  6. 10. 15. 21. 25. 27. 27. 25. 21. 15. 10.  6.  3.  1.]


In [18]:
#moving on to the tougher version
def freq_dict(some_dict):
    # Faces is just the set of different faces stored in a list
    faces = sorted(some_dict.keys())[::-1]
    size = sum(key*some_dict[key] for key in some_dict) - sum(some_dict[key] for key in some_dict) + 1
    # freq_matrix will store the results, I will keep it stored in the same place 
    freq_matrix = np.zeros(size)
    freq_matrix[:faces[0]] = 1 
    # The transformation matrix will change slightly for each face 
    # To start, initialize to the largest die number and we will adjust as we go
    A = np.ones((size,size))
    A = np.triu(A)
    A = np.triu(np.flip(A,1))
    for i in range(size-2*faces[0]):
        A[i][faces[0]+i:size-(faces[0]+i)] = 0
    
    # Handle the first size dice to get the ball rollin
    if some_dict[faces[0]] > 1:
        freq_matrix = np.linalg.matrix_power(A.T,some_dict[faces[0]]-1).dot(freq_matrix)
    
    # Now handle the rest of the dice
    for face in faces[1:]:
        for i in range(size-2*face):
            A[i][face+i:size-(face+i)] = 0
        freq_matrix = np.linalg.matrix_power(A.T,some_dict[face]).dot(freq_matrix)
    
    return freq_matrix
    

In [24]:
test = {10:10,6:2,4:2,3:3}
print(freq_dict(test) )

[1.00000000e+00 1.70000000e+01 1.53000000e+02 9.66000000e+02
 4.79200000e+03 1.98560000e+04 7.14010000e+04 2.28707000e+05
 6.64990000e+05 1.78034700e+06 4.43792300e+06 1.03921140e+07
 2.30263930e+07 4.85680170e+07 9.80068400e+07 1.90013011e+08
 3.55217924e+08 6.42285463e+08 1.12624386e+09 1.91955667e+09
 3.18636684e+09 5.16023183e+09 8.16546444e+09 1.26418903e+10
 1.91724258e+10 2.85123743e+10 4.16187618e+10 5.96774098e+10
 8.41248515e+10 1.16661679e+11 1.59253573e+11 2.14116159e+11
 2.83680101e+11 3.70533426e+11 4.77339155e+11 6.06727770e+11
 7.61165901e+11 9.42804727e+11 1.15331388e+12 1.39370880e+12
 1.66418151e+12 1.96394621e+12 2.29111184e+12 2.64259371e+12
 3.01407529e+12 3.40002906e+12 3.79380227e+12 4.18776990e+12
 4.57355232e+12 4.94229075e+12 5.28496915e+12 5.59276701e+12
 5.85742454e+12 6.07159985e+12 6.22919715e+12 6.32564621e+12
 6.35811556e+12 6.32564621e+12 6.22919715e+12 6.07159985e+12
 5.85742454e+12 5.59276701e+12 5.28496915e+12 4.94229075e+12
 4.57355232e+12 4.187769

In [22]:
get_dict_freq({10:10,6:2,4:2, 3:3})

array([            0,             0,             0,             0,
                   0,             0,             0,             0,
                   0,             0,             0,             0,
                   0,             0,             0,             0,
                   0,             1,            17,           153,
                 966,          4792,         19856,         71401,
              228707,        664990,       1780347,       4437923,
            10392114,      23026393,      48568017,      98006840,
           190013011,     355217924,     642285463,    1126243856,
          1919556669,    3186366837,    5160231826,    8165464439,
         12641890335,   19172425758,   28512374343,   41618761773,
         59677409760,   84124851504,  116661679422,  159253572549,
        214116158565,  283680100941,  370533426288,  477339155256,
        606727770480,  761165900832,  942804727272, 1153313878152,
       1393708795800, 1664181508626, 1963946211954, 2291111837